In [16]:
import sys
sys.path.append("r/C:/Users/tlnguyen3/Anaconda3/envs/snakes/Lib/site-packages")
import cplex
from cplex.callbacks import LazyConstraintCallback
import numpy as np
from numpy import random
import networkx as nx

# num_constraints_sum_arcs = len(E)
# num_constraints_assignments = len(E) * 2
def sol_SEP(x_star, edge2idx):
    Problem = cplex.Cplex()

    Xedge2idx = {}
    Xidx2edge = {}
    
    num=0
    for edge in G.edges:
        i = edge[0]
        j = edge[1]
        if i > edge[1]:
            i = edge[1]
            j = edge[0]
        var_name = [f"x.{i}.{j}",
                    f"x.{j}.{i}",
                    f"y.{i}.{j}",
                   ]
        var_index = Problem.variables.add(
            obj=[0.0,0.0,1.0],
            lb=[0.0,0.0,0.0],
    #         ub=[0.0,0.0,0.0],
            types=["C","C","C"],
            names=var_name,
        )
        Xedge2idx[edge] = var_index
        Xidx2edge[var_index] = edge

    #     print(x_star[i])
        Problem.linear_constraints.add(
            lin_expr= [cplex.SparsePair(ind = var_index[:2], 
                                        val= [1.0, 1.0], 
                                       )
                      ],
                        rhs= [x_star[edge2idx[(i, j)]]],
                        senses = ["E"],
                        names = [f"c{num}"],
            )
        num = num+1
    # =======================================
    #print("den buoc nay")
    num1=0
    for edge in G.edges:
        i = edge[0]
        j = edge[1]
        if i > edge[1]:
            i = edge[1]
            j = edge[0]
        #k=0
        neighbor_i=[]
        for k in G.neighbors(i):
            if k != j:
                if i <= k:
                    neighbor_i.append(Xedge2idx[(i, k)][0])
                else:
                    neighbor_i.append(Xedge2idx[(k, i)][1])
        #neighbor_i = [Xedge2idx[(i, k)][:2] if k != j for k in G.neighbors(i)]
        neighbor_i.append(Xedge2idx[(i, j)][2])
        coefs = np.ones_like(neighbor_i, dtype=float)
        coefs[-1] = -1.
#         print("vao vong lap")
#         print("x_star[edge2idx[(i, j)]]", x_star[edge2idx[(i, j)]])
        Problem.linear_constraints.add(
        lin_expr= [cplex.SparsePair(ind = neighbor_i, 
                                    val= coefs, 
                                   )
                  ],
                    rhs= [1-x_star[edge2idx[(i, j)]]],
                    senses = ["L"],
                    names = [f"d{num1}"],
        )
        num1 = num1 + 1
        #print("da add them constraint", num1)
#     =======================================
    #print("den buoc num 2")
    num2 = 0
    for edge in G.edges:
        i = edge[0]
        j = edge[1]
        if i< edge[1]:
            i= edge[1]
            j = edge[0]
        neighbor_i= []
        for k in G.neighbors(i):
            if k != j:
                if i <= k:
                    neighbor_i.append(Xedge2idx[(i, k)][0])
                else:
                    neighbor_i.append(Xedge2idx[(k, i)][1])
#         print(neighbor_i)
#         print("abc")
#         print("print(Xedge2idx[(i, j)][2]) is", Xedge2idx[(j, i)][2])
        neighbor_i.append(Xedge2idx[(j, i)][2])
#         print("neighbor", neighbor_i)
        coefs = np.ones_like(neighbor_i, dtype=float)
        coefs[-1] = -1.
#         print("vao vong lap thu 2")
#         print("x_star[edge2idx[(i, j)]]", x_star[edge2idx[(j, i)]])
        
        Problem.linear_constraints.add(
        lin_expr= [cplex.SparsePair(ind = neighbor_i, 
                                    val= coefs, 
                                   )
                  ],
                    rhs= [1-x_star[edge2idx[(j, i)]]],
                    senses = ["L"],
                    names = [f"e{num2}"],
        )
        num2 = num2 + 1 
        
    print("finish to construct")
    Problem.write("lpex1.lp")
    Problem.objective.set_sense(Problem.objective.sense.minimize)
    error = Problem.solve()
    if not error:
        print('Solution',Problem.solution.get_values())
        print('Optimal',Problem.solution.get_objective_value())
        return Problem.solution.get_objective_value()
    else:
        print('Sorry')
    return Problem.solution.get_values()
    
        

# G = nx.Graph()
# G = nx.complete_graph(12)
# G.add_weighted_edges_from((u, v, random.random()) for u, v in G.edges)
# # x_star = [random.choice([0., 1.]) for k in range(len(G.edges))]
# x_sol = sol_SEP(x_star)
# if x_sol != None:
#     print('found',x_sol)
#     if x_sol == 0:
#         print('and it is good')


In [17]:
def findPaths(G, u, n):
    if n == 0:
        return [[u]]
    paths = [[u] + path for neighbor in G.neighbors(u) for path in findPaths(G, neighbor, n - 1)
             if u not in path]
    return paths

def findCycles(G, u, n):
    c = findPaths(G, u, n - 1)
    for cycle in c:
        if cycle[0] in G.neighbors(cycle[n - 1]):
            cycle.extend([cycle[0]])
    return c

class UserLazyConstraints(LazyConstraintCallback):
    def __call__(self):
        print(self.get_values())
        x_star = self.get_values()
        x_sol = sol_SEP(x_star, self.edge2idx)
        print("get x_sol",x_sol)
        for path in self.allpc:
            edges = []
            for i in range(len(path) - 1):
                edges.append((min(path[i], path[i+1]), max(path[i], path[i+1])))
                
            values = []
            for edge in edges:
                values.append(x_star[self.edge2idx[edge]])
                
            if sum(values) > 2 +1e-3 :
                cut = cplex.SparsePair(ind=["x.{}.{}".format(*edges[0]),
                                            "x.{}.{}".format(*edges[1]),
                                           "x.{}.{}".format(*edges[2])],
                                          val=[1.0] * len(edges))
                self.add(constraint= cut, sense="L", rhs=2.0)
                
if __name__ == "__main__":
    G = nx.Graph()
    G = nx.complete_graph(11)
    G.add_weighted_edges_from((u, v, random.random()) for u, v in G.edges())

    allpaths = []
    for node in G:
        allpaths.extend(findPaths(G, node, 3))
    allcycles = []
    for node in G:
        allcycles.extend(findCycles(G, node, 3))
    allpc = allcycles + allpaths
    num_constraints = len (allpc)
    
    Problem = cplex.Cplex()

    edge2idx = {}
    idx2edge = {}
    for edge in G.edges:
        weight = G.edges[edge]["weight"]
        var_name = "x.{}.{}".format(*edge)
        var_index = Problem.variables.add(
            obj=[weight],
            lb=[0.0],
            ub=[1.0],
            types=["B"],
            names=[var_name],
        )[0]
        edge2idx[edge] = var_index
        idx2edge[var_index] = edge
    Problem.write("lpex2.lp")
    Problem.objective.set_sense(Problem.objective.sense.maximize)
    lazy = Problem.register_callback(UserLazyConstraints)
    lazy.G = G
    lazy.allpc = allpc
    lazy.edge2idx = edge2idx
    Problem.solve()
    print(Problem.solution.get_values)
    #x_star = Problem.solution.get_values()
    #x_sol = sol_SEP(x_star, edge2idx)
    #print(x_sol)

Default row names c1, c2 ... being created.


CPXPARAM_Read_DataCheck                          1


Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
Tried aggregator 1 time.
Reduced MIP has 0 rows, 55 columns, and 0 nonzeros.
Reduced MIP has 55 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.01 ticks)


[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
finish to construct
CPXPARAM_Read_DataCheck                          1


CPLEX Error  1017: Not available for mixed-integer problems.


...

CPLEX Error  1017: Not available for mixed-integer problems.


CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.02 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.02 ticks)
Solution [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

finish to construct
CPXPARAM_Read_DataCheck                          1


CPLEX Error  1017: Not available for mixed-integer problems.


CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.02 ticks)

Root node processing (before b&c):
  Real time             =    0.00 sec. (0.02 ticks)
Parallel b&c, 12 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.00 sec. (0.02 ticks)
Solution [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0